In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
import tensorflow as tf
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [ ]:
data = pd.read_csv("/kaggle/input/fraud-detection-datafest/Copy of FraudDetectionDataset.csv")

# **Data Visualization and Feature Engineering**

In [ ]:
numeric_features  =['Transaction ID','User ID','Transaction Amount',
                    'Merchant ID','User Age','User Income','Location Distance',
                    'Time Taken for Transaction',"User's Transaction History",
                    "Merchant's Reputation Score","User's Credit Score",
                    "Merchant's Business Age"] 

all_features = ['Transaction ID', 'User ID', 'Transaction Amount',
       'Transaction Date and Time', 'Merchant ID', 'Payment Method',
       'Country Code', 'Transaction Type', 'Device Type',
       'Browser Type', 'Operating System', 'Merchant Category', 'User Age',
       'User Occupation', 'User Income', 'User Gender', 'User Account Status',
       'Transaction Status', 'Location Distance', 'Time Taken for Transaction',
       'Transaction Time of Day', "User's Transaction History",
       "Merchant's Reputation Score", "User's Device Location",
       'Transaction Currency', 'Transaction Purpose', "User's Credit Score",
       "User's Email Domain", "Merchant's Business Age",
       'Transaction Authentication Method']
categorical_features = list(set(all_features)-set(numeric_features))


In [ ]:
data[numeric_features].describe()

In [ ]:
data[categorical_features].describe()

In [ ]:
sns.countplot(data=data, x="Fraudulent Flag")
plt.show()

In [ ]:
#converting the  categorical variables to numbers

def convert(data,columns):
    allmap ={}
    for i in columns:
        unique = list(data[i].unique())
        mapdict ={}
        for p,u in enumerate(unique):
            mapdict[u]=p 
        data[i]=data[i].map(mapdict)
        print("done with:{}".format(i))
        allmap[i]= mapdict
    return mapdict
lookup_table= convert(data,categorical_features)

In [ ]:
for i, q in enumerate(categorical_features[0:10]):
    plt.figure(figsize=(20,7))
    sns.scatterplot(data=data, x=q, y="Fraudulent Flag")
    print (i)
    plt.show()

In [ ]:
# investigating categories in fradulent transactions

for i, q in enumerate(categorical_features[0:12]):
    plt.figure(figsize=(20,7))
    dat_fraud =data[(data["Fraudulent Flag"]==1)][q]
    sns.distplot(dat_fraud )
    print (i)
    plt.show()

**From all indications this the Fradulent cases appear to be uniformly distrbuted between the various classes in the categorical columns of the dataset with almost no glaring predictors**

In [ ]:
# investigating categories in non-fradulent transactions

for i, q in enumerate(categorical_features[0:12]):
    plt.figure(figsize=(20,7))
    dat_fraud =data[(data["Fraudulent Flag"]==0)][q]
    sns.distplot(dat_fraud )
    print (i)
    plt.show()

**From all indications this the non fradulent cases appear to be uniformly distrbuted between the various classes in the categorical columns just like we saw in the fradulent one**

# Data preparation and categorical preprocessing with Target Encoding 

In [ ]:
from category_encoders import TargetEncoder
for cla in categorical_features:
    encoder = TargetEncoder()
    data[cla] = encoder.fit_transform(data[cla], data['Fraudulent Flag'])

# Model Training and Evaluation

 ## Approach 1- Plain Logistic Regression classifier

In [ ]:
#the Data wil be scaled using Standard Scaler and Dimensionality reduction will be carried out using Princpal Componet Analysis 
pca = PCA(n_components=15)
x= data[all_features]
Scaler = StandardScaler()
x = Scaler.fit_transform(x)
x=pca.fit_transform(x)
y= data["Fraudulent Flag"].values

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state =27)
clf=LogisticRegression(solver ="lbfgs").fit(x_train,y_train)

y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred))

fig, ax =plt.subplots()
sns.heatmap(confusion_matrix(y_test,y_pred, normalize='true'), annot =True,ax= ax)
ax.set_title("Confusion Matrix")
ax.set_ylabel("Real value")
ax.set_xlabel("predicted value")
plt.show()

With this model we get a model that's about 82% accurate at identifying both legitimate and fraudulent transactions